In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import patsy as pt
import statsmodels.api as sm
from sklearn.linear_model import Ridge

In [3]:
df = pd.read_csv(r'661.csv')

In [4]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### questao 1 -qual a media da variavel carat?

In [5]:
df.describe()
#media igual a 0.7979

,Unnamed: 0,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,26970.500000,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,15571.281097,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,1.000000,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,13485.750000,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,26970.500000,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,40455.250000,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,53940.000000,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [6]:
# deletamos a coluna Unnamed: 0 pois se assemelha ao index
df = df.drop(columns='Unnamed: 0')

### questão 2-Qual é a correlação entre as variáveis ‘price’ e ‘depth’?

In [7]:
df.corr()
#correlação entre elas é -0.010647


,carat,depth,table,price,x,y,z
carat,1.000000,0.028224,0.181618,0.921591,0.975094,0.951722,0.953387
depth,0.028224,1.000000,-0.295779,-0.010647,-0.025289,-0.029341,0.094924
table,0.181618,-0.295779,1.000000,0.127134,0.195344,0.183760,0.150929
price,0.921591,-0.010647,0.127134,1.000000,0.884435,0.865421,0.861249
x,0.975094,-0.025289,0.195344,0.884435,1.000000,0.974701,0.970772
y,0.951722,-0.029341,0.183760,0.865421,0.974701,1.000000,0.952006
z,0.953387,0.094924,0.150929,0.861249,0.970772,0.952006,1.000000


### Questão 3 - Após o escalonamento das variáveis numéricas, qual é o valor máximo da variável ‘table’?

In [8]:
# separamos o df em numerico e categorico
dfNum = df.select_dtypes(include='number').drop(columns='price') #df numerico
# transformamos as categorias str em categorias objeto
df[["cut",'color','clarity']]=df[["cut",'color','clarity']].astype('category')
dfCat= df.select_dtypes('category')

In [15]:
# criamos uma funçao lambda para escalonar 
# o escalonamento é feio com o valor da variavel menos a media dividido pelo desvio padrao(com seus graus de liberdade)
df_norm=dfNum.apply(lambda x: (x-(np.mean(x)))/np.std(x,ddof=1))
df_norm.max()
# maximo da variavel table é 16.80 apos o escalonamento

carat     8.885992
depth    12.041280
table    16.801510
x         4.465161
y        46.549216
z        40.047205
dtype: float64

In [16]:
# fizemos a dumificação das variaveis em dfCat e unimos ao df normalizado
dfCat=pd.get_dummies(dfCat,drop_first=True)
df_final = df_norm.merge(dfCat,how='left',right_index=True,left_index=True)
df_final

,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,-1.198157,-0.174090,-1.099662,-1.587823,-1.536181,-1.571115,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,-1.240350,-1.360726,1.585514,-1.641310,-1.658759,-1.741159,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,-1.198157,-3.384987,3.375631,-1.498677,-1.457382,-1.741159,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,-1.071577,0.454129,0.242926,-1.364959,-1.317293,-1.287708,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
4,-1.029384,1.082348,0.242926,-1.240155,-1.212227,-1.117663,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,-0.164426,-0.662705,-0.204603,0.016798,0.022304,-0.054887,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
53936,-0.164426,0.942744,-1.099662,-0.036690,0.013548,0.100987,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
53937,-0.206619,0.733338,1.137985,-0.063434,-0.047740,0.030135,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
53938,0.130926,-0.523100,0.242926,0.373380,0.337503,0.285201,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0


##### Questão 4 - Ao separar o dataset em dois pedaços, o de treinamento e o de teste, quantas linhas possuirá o dataset de teste? (Utilize uma proporção de 20% e argumento random_state = 123456) Dica: concatene os datasets e faça a matriz do modelo antes de fazer a separação entre treinamento e teste

In [20]:
#criamos a variavel x com os valores previsores
x=df_final
#criamos a variavel y com os valores que deveram ser previsto
y =df['price']
# usamos a função train_test_split para dividir o dataset de treino e de test com um test de tamanho 20% do dataset
xtrain, xtest, ytrain, ytest= train_test_split(x,y,test_size=0.2, random_state=123456)
xtest# vemos que xtest possui 10788 linhas

,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
1277,-0.185522,0.454129,-0.204603,-0.045604,-0.003963,0.030135,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
22475,1.586587,-0.872111,1.137985,1.567930,1.510745,1.390489,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
40809,-0.607453,1.291755,0.690455,-0.535905,-0.590584,-0.423316,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
22773,0.447374,-0.802309,-0.652132,0.712133,0.652702,0.568608,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
41414,-0.691840,-0.732507,-0.607380,-0.633965,-0.573072,-0.664212,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35729,-1.050481,-0.034486,0.242926,-1.257984,-1.256004,-1.245197,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
8405,0.573953,0.803140,0.690455,0.756706,0.678969,0.823675,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
5300,0.447374,-0.104288,-1.099662,0.685389,0.626436,0.639460,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
51088,-0.206619,-1.221122,-0.204603,0.007883,0.057326,-0.111569,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0


### Questão 5 - Utilizando o método dos mínimos quadrados e ajustando o modelo com o logaritmo natural da resposta ‘price’ e os dados de ajuste, qual é a variável que tem o maior valor absoluto para a estatística ‘t’. (OBS: desconsidere o ‘intercept’).

In [21]:
dfFull = pd.concat([y, df_final], axis = 1)

In [22]:
dfFull['cut_Very_Good']=dfFull['cut_Very Good']

In [23]:
dfFull=dfFull.drop(columns='cut_Very Good')

In [24]:
def ols_formula(df, dependent_var, *excluded_rows):
    # Listar o nome das colunas do dataframe
    dfCols = list(df.columns.values)
    # Remover a variável dependente
    dfCols.remove(dependent_var)
    # Remover as variáveis excluídas
    for col in excluded_rows:
        dfCols.remove(col)
    # Retornar a fórmula
    return dependent_var + ' ~ ' + ' + '.join(dfCols)

In [25]:
dfFull['price']=np.log(df['price'])

In [26]:
y, X = pt.dmatrices(ols_formula(dfFull, 'price'), data = dfFull , return_type='dataframe')

In [27]:
# Especificação do modelo
reg1 = sm.OLS(y, X)

# Executa o método dos mínimos quadrados
reg1fit = reg1.fit()
print(reg1fit.summary())
# dentre as respostas a x apresenta maior estatistica t

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                 7.597e+04
Date:                Fri, 22 Jul 2022   Prob (F-statistic):               0.00
Time:                        14:11:18   Log-Likelihood:                 17312.
No. Observations:               53940   AIC:                        -3.458e+04
Df Residuals:                   53916   BIC:                        -3.436e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         7.1159      0.008    907.657

### Questão 6 - Para o modelo ajustado na questão 05, qual é a variável que possui o maior VIF?

In [28]:
#calculei o vif para X
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)
# o maior vif esta na variavel X


          feature         VIF
0       Intercept  107.542475
1           carat   22.439582
2           depth    1.782401
3           table    1.787765
4               x   57.518327
5               y   20.592160
6               z   23.585582
7        cut_Good    3.940574
8       cut_Ideal   11.308678
9     cut_Premium    8.348051
10        color_E    2.009954
11        color_F    2.013118
12        color_G    2.194151
13        color_H    1.951752
14        color_I    1.710211
15        color_J    1.423173
16     clarity_IF    3.527903
17    clarity_SI1   14.759699
18    clarity_SI2   11.466553
19    clarity_VS1   10.772677
20    clarity_VS2   14.263817
21   clarity_VVS1    5.933726
22   clarity_VVS2    7.557383
23  cut_Very_Good    7.631051


### Questão 7 - Após remover todas as variáveis para manter os VIF´s em no máximo 5, quantas variáveis sobraram no modelo? OBS: desconsidere o intercept. (Dica, comece removendo, uma por uma, as variáveis com maior VIF)


desconsideramos as variaveis dumificadas e o intercept e as variaveis com vif acima de 5 ficando apenas com depht table somado as  colunas dumificadas 
totalizando 19

### Questão 8 - Qual é o R^2 para os dados de teste com o modelo sem as variáveis removidas na questão 7?


In [29]:
df_ajustado = df_norm
df_ajustado=df_ajustado[['depth','table']]
df_ajustado = pd.concat((df_ajustado,dfCat),axis=1)
df_ajustado['price'] =np.log(df['price'])
df_ajustado

,depth,table,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,price
0,-0.174090,-1.099662,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,5.786897
1,-1.360726,1.585514,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,5.786897
2,-3.384987,3.375631,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,5.789960
3,0.454129,0.242926,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,5.811141
4,1.082348,0.242926,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,5.814131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,-0.662705,-0.204603,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,7.921898
53936,0.942744,-1.099662,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,7.921898
53937,0.733338,1.137985,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,7.921898
53938,-0.523100,0.242926,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,7.921898


In [30]:
df_ajustado['cut_Very_Good']=df_ajustado['cut_Very Good']
df_ajustado = df_ajustado.drop(columns='cut_Very Good')

In [31]:
# ajustando novamente
y, X = pt.dmatrices(ols_formula(df_ajustado, 'price'), data = df_ajustado , return_type='dataframe')

In [33]:
reg2=sm.OLS(y,X)
reg2fit=reg2.fit()
reg2fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     294.9
Date:                Fri, 22 Jul 2022   Prob (F-statistic):               0.00
Time:                        14:12:03   Log-Likelihood:                -74655.
No. Observations:               53940   AIC:                         1.494e+05
Df Residuals:                   53920   BIC:                         1.495e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         7.7427      0.043    180.041      0.000       7.658       7.827
depth             0.0274      0.005      5.624      0.000       0.018       0.037
table             0.1281      0.006     23.142      0.000       0.117       0.139
cut_Good         -0.1149      0.029     -4.007      0.000      -0.171      -0.059
cut_Ideal        -0.0463      0.029     -1.625      0.104      -0.102       0.010
cut_Premium       0.0118      0.028      0.430      0.667      -0.042       0.066
color_E          -0.0084      0.015     -0.548      0.583      -0.038       0.022
color_F           0.1930      0.015     12.522      0.000       0.163       0.223
color_G           0.2599      0.015     17.290      0.000       0.230       0.289
color_H           0.3168      0.016     19.952      0.000       0.286       0.348
color_I           0.4242      0.018     24.019      0.000       0.390       0.459
color_J           0.5086      0.022     23.385      0.000       0.466       0.551
clarity_IF       -0.4762      0.043    -11.065      0.000      -0.561      -0.392
clarity_SI1      -0.0681      0.037     -1.836      0.066      -0.141       0.005
clarity_SI2       0.2345      0.037      6.272      0.000       0.161       0.308
clarity_VS1      -0.1971      0.038     -5.224      0.000      -0.271      -0.123
clarity_VS2      -0.1354      0.037     -3.639      0.000      -0.208      -0.062
clarity_VVS1     -0.5525      0.040    -13.918      0.000      -0.630      -0.475
clarity_VVS2     -0.3341      0.039     -8.622      0.000      -0.410      -0.258
cut_Very_Good    -0.0587      0.028     -2.133      0.033      -0.113      -0.005
==============================================================================
Omnibus:                     2916.320   Durbin-Watson:                   0.152
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1195.376
Skew:                           0.107   Prob(JB):                    2.68e-260
Kurtosis:                       2.303   Cond. No.                         31.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""